In [1]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

import ast
import math
import operator
import mmap

import gzip
import gensim
import logging
import os
import ast

And now we have the if file, we can make our query algorithm, the TA (threshould) algorithm, for this we need first to define some functions

In [2]:
def sorted_access(list, index, T):
    if len(list[index]) == 0:
        return None, None, T
    else:
        id, score = next(iter(list[index].items()))
        del list[index][id]
    return id, score, score


def random_access(list, index, id, T):
    if len(list[index]) == 0:
        return 0, 0
    elif id not in list[index]:
        return 0, T
    else:
        score = list[index][id]
        if id == next(iter(list[index].keys())):
            T = score
        del list[index][id]
        return score, T

we can try it on the example of the course

In [3]:
list = [{2:0.9,5:0.8,6:0.7,4:0.6,1:0.5,3:0.4},{3:0.85,5:0.8,2:0.75,6:0.74,1:0.74,4:0.7}]
tokens = len(list)
C = {}
T = [math.inf] * tokens
k=3

# start of the algorithm
while (len(C) < k) or not all(score >= sum(T) / tokens for score in C.values()):
    same = True

    for i in range(0, tokens):
        id, score, T[i] = sorted_access(list, i, T[i])

        if id is not None:
            same = False
            C[id] = score

            for j in [x for x in range(0, tokens) if x != i]:
                score, T[j] = random_access(list, j, id, T[j])
                C[id] = C[id] + score

            C[id] = C[id] / tokens
            if len(C) > k:
                C = dict(sorted(C.items(), key=operator.itemgetter(1), reverse=True))
                C.popitem()

    if same:
        print("same",C)
        exit()

print (C)

{2: 0.825, 5: 0.8, 6: 0.72}


and finally the real threshold algorithm with all the functions needed, and the support of "or" and "and" queries

In [4]:
def access_line(file, index):
    with open(file, "r+") as f:
        mm = mmap.mmap(f.fileno(), 0)
        mm.seek(index)
        return mm.readline().decode().replace('\\','')

In [5]:
def load_voc(filename):
    with open(filename,"r") as file:
        voc = ast.literal_eval(file.readline())
    return voc

In [6]:
def load_tokens(query, voc, file):
    stop_words = stopwords.words('english')
    stop_words.extend(["the"])
    stop_words = set(stop_words)
    stemmer = SnowballStemmer('english')
    query = re.sub(r'[^\w]', ' ', query)
    tokens = word_tokenize(re.sub(r"\d+", "", query.lower()))
    tokens = [w for w in tokens if w not in stop_words]
    tokens = set([stemmer.stem(w) for w in tokens])
    list = [dict(ast.literal_eval(access_line(file, voc[token]))) for token in tokens if token in voc.keys()]
    return list


def filter_all(list):
    index = 0
    minimum = len(list[0])
    for l in range(1, len(list)):
        if len(list[l]) < minimum:
            minimum = len(list[l])
            index = l
    for i in [x for x in range(0, len(list)) if x != index]:
        delete = [key for key in list[index] if key not in list[i]]
        for key in delete: del list[index][key]
    for i in [x for x in range(0, len(list)) if x != index]:
        delete = [key for key in list[i] if key not in list[index]]
        for key in delete: del list[i][key]
    return list

In [7]:
def ta(query, k=3, all_terms=0):
    # loading the lis    
    voc = load_voc("if_access.csv")
    list = load_tokens(query, voc, "if.csv")
    tokens = len(list)
    if tokens == 0:
        return None
    if all_terms:
        list = filter_all(list)

    # initialize the parameters
    C = {}
    T = [math.inf] * tokens

    # start of the algorithm
    while (len(C) < k) or not all(score >= sum(T) / tokens for score in C.values()):
        same = True

        for i in range(0, tokens):
            id, score, T[i] = sorted_access(list, i, T[i])

            if id is not None:
                same = False
                C[id] = score

                for j in [x for x in range(0, tokens) if x != i]:
                    score, T[j] = random_access(list, j, id, T[j])
                    C[id] = C[id] + score

                C[id] = C[id] / tokens
                if len(C) > k:
                    C = dict(sorted(C.items(), key=operator.itemgetter(1), reverse=True))
                    C.popitem()

        if same:
            return (C)

    return (C)

we can test it on this example for "or" and "and"

In [8]:
ta("inventory organization",3,0)

{106803: 9.682111973959575, 6953: 9.612125479565474, 83602: 9.241174928546245}

In [9]:
ta("inventory organization",3,1)

{106803: 9.682111973959575,
 282354: 8.538468979317457,
 11662: 7.338185328887775}

and the same result even like that

In [10]:
ta("The organized-inventory   ORGANIZATION",3,0)

{106803: 9.682111973959575, 6953: 9.612125479565474, 83602: 9.241174928546245}

In [11]:
ta("The organized-inventory   ORGANIZATION",3,1)

{106803: 9.682111973959575,
 282354: 8.538468979317457,
 11662: 7.338185328887775}

Now we just need a small function to retireve all the articles of a query

In [12]:
def load_articles(list):
    articles_list = load_voc("filtered_access.csv")
    for id, score in list.items():
        print(id,": ",score)
        print(access_line("filtered.csv", articles_list[str(id)]).split("#")[1])

In [13]:
load_articles(ta("inventory organization",3,0))

106803 :  9.682111973959575
PANEL ALLEGES 'MASSIVE FRAUD' COMMITTED BY EX-MINISCRIBE OFFICIALS The former managers of computer disk-drive maker MiniScribe Corp. allegedly "perpetrated a massive fraud" by masking the company's declining fortunes with elaborately faked financial statements, an independent panel charged Tuesday. The panel, made up of newly elected directors and auditors, described unusual schemes to boost sales and inventory as the business fell on hard times. In one instance, the company shipped bricks to distributors and marked them down as computer component sales. In another, repackaged scrap metal and obsolete parts were made to appear to be new products. "The fraud was all the more alarming in that it required the active participation of many company personnel, and was apparently common knowledge among a great many other employees," said the panel, which submitted its study of the business practices of Longmont, Colo.-based MiniScribe to the Securities and Exchange 

Finally we are going to define a function that provide documents that are similar to the top k obtained, for this we are going first to restrict to the documents that contains at least one of the terms, than calculate their similarity using jaccard similarity which is independent from the size of the document

In [14]:
def get_articles(list, articles):
    results = set()
    for i in range(len(list)):
        for doc in list[i].keys():
            if doc not in articles:
                results.add(doc)
    return results

In [15]:
def jaccard(article, other, list):
    union = len(list[article].union(list[other]))
    intersect = len([w for w in list[article] if w in list[other]])
    if union == 0:
        return 0
    else:
        return (intersect / union)

In [16]:
def search_articles(query, threshold, k=3, all=0,limit = 0):
    articles = ta(query,k,all)
    
    voc = load_voc("if_access.csv")

    if limit > k:
        limit = limit - k
        other_articles = load_tokens(query, voc, "if.csv")
        other_articles = get_articles(other_articles, articles.keys())
        
        index = load_voc("tokens_access.csv")
        
        list = {}
        
        for article in articles.keys():
            list[article] = set(ast.literal_eval(access_line("tokens.csv", index[str(article)]).split('#')[1][1:-3]))
        
        for other in other_articles:
            list[other] = set(ast.literal_eval(access_line("tokens.csv", index[str(other)]).split('#')[1][1:-3]))
        
        score = 0.0
        results = {}
        for article in articles.keys():
            for other in other_articles:
                score = jaccard(article, other, list)
                if score >= threshold:
                    results[other] = score
            other_articles = [doc for doc in other_articles if doc not in results.keys()]

        for key, value in results.items():
            articles[key] = value
            limit-=1
            if limit == 0:
                return articles    
    return articles

here we can see that the document 6953 which is the top 3 is close to 6681

In [17]:
load_articles(search_articles("inventory organization",0.6,3,0,10))

106803 :  9.682111973959575
PANEL ALLEGES 'MASSIVE FRAUD' COMMITTED BY EX-MINISCRIBE OFFICIALS The former managers of computer disk-drive maker MiniScribe Corp. allegedly "perpetrated a massive fraud" by masking the company's declining fortunes with elaborately faked financial statements, an independent panel charged Tuesday. The panel, made up of newly elected directors and auditors, described unusual schemes to boost sales and inventory as the business fell on hard times. In one instance, the company shipped bricks to distributors and marked them down as computer component sales. In another, repackaged scrap metal and obsolete parts were made to appear to be new products. "The fraud was all the more alarming in that it required the active participation of many company personnel, and was apparently common knowledge among a great many other employees," said the panel, which submitted its study of the business practices of Longmont, Colo.-based MiniScribe to the Securities and Exchange 

Now to predict the closest words in meaning to those on the query, we are going to train a small neural network with just one hidden layer, Word2Vec

In [18]:
def load_data(filename):
    data = []
    with open(filename,"r") as file:
        for line in file:
            res = ast.literal_eval(line.split("#")[1][1:-2])
            if len(res) > 0:
                data.append(res)
    return data

In [19]:
documents = load_data("tokens.csv")

model = gensim.models.Word2Vec(
    documents,
    size=150,
    window=10,
    min_count=2,
    workers=10)
model.train(documents, total_examples=len(documents), epochs=100)

model.save("word2vec.model")

And we can test it here with the same example

In [20]:
model = gensim.models.Word2Vec.load("word2vec.model")

query = "inventory organization"
voc = load_voc("if_access.csv")
stop_words = stopwords.words('english')
stop_words.extend(["the"])
stop_words = set(stop_words)
stemmer = SnowballStemmer('english')
query = re.sub(r'[^\w]', ' ', query)
tokens = word_tokenize(re.sub(r"\d+", "", query.lower()))
tokens = [w for w in tokens if w not in stop_words]
tokens = set([stemmer.stem(w) for w in tokens])
query = [token for token in tokens if token in voc.keys()]

print("Most similar to {0}".format(query), model.wv.most_similar(positive=query, topn=5))

Most similar to ['inventori', 'organ'] [('group', 0.5001776814460754), ('membership', 0.48198387026786804), ('sector', 0.4637340307235718), ('sale', 0.4511248469352722), ('industri', 0.44904905557632446)]
